## Making Sensors

A short tutorial on making sensors with observations

In [1]:
#imports

import pyshdom
import numpy as np
import xarray as xr
from collections import OrderedDict
import matplotlib.pyplot as plt

### The default sensor generator

The basic sensor generator (`pyshdom.sensor.make_sensor_dataset`)takes `np.array`'s of the angles and positions of pixels. It makes no assumptions about the FOV of the pixels. A possible default behaviour is to assume that each pixel is modeled by a single infinitesmal ray at the pixel's location and pointing direction. This default behaviour can be chosen by selecting `fill_ray_variables=True`. Note that ray variables must be defined in the sensor for it to be valid. So you must use this option or generate the rays from the pixels according to a model.

The different sensor geometry generators within `pyshdom.sensor` make use of `pyshdom.sensor.make_sensor_dataset` but apply their own model for the ray variables and generation of pixel variables.

#### Observables:

Each sensor has some observables which are its spectral parameter `wavelength` and the components of the `stokes` vector. 

Note that `wavelength` just must be a unique identifier for a spectral channel and is used to match sensors with optical properties and RTE solvers. It is not necessarily a monochromatic wavelength though that is the typical use case as we have no explicit representation of narrow/broad band sensors.
Sensors which have a spectral response function and their observables are weighted averages over different wavelengths require explicit post processing by the user. In the inverse problem use of spectrally averaged observables must be modified through definition of a new objective function in `pyshdom.gradient`.

The `stokes` components can be (I, Q, U, V) with a minimum of I.


In [4]:
mu = phi = x = y = z = np.array([1.0])
stokes = ['I']
wavelength = 0.86
sensor = pyshdom.sensor.make_sensor_dataset(x, y, z, mu, phi, stokes, wavelength)
sensor

<xarray.Dataset>
Dimensions:       (stokes_index: 4, npixels: 1)
Coordinates:
  * stokes_index  (stokes_index) <U1 'I' 'Q' 'U' 'V'
Dimensions without coordinates: npixels
Data variables:
    wavelength    float64 0.86
    stokes        (stokes_index) bool True False False False
    cam_x         (npixels) float64 1.0
    cam_y         (npixels) float64 1.0
    cam_z         (npixels) float64 1.0
    cam_mu        (npixels) float64 1.0
    cam_phi       (npixels) float64 1.0

`sensor` is not valid unless we include the ray variables. 

The default behaviour is to not include the ray variables because I want to force people to be aware of the choice they are making about how each pixel's FOV is modeled, which can have significant implications when these measurements are interpreted or used in the inverse problem.

In [5]:
pyshdom.checks.check_sensor(sensor)

KeyError: "Expected variable with name 'ray_mu' in dataset"

In [6]:
# Now it works
sensor = pyshdom.sensor.make_sensor_dataset(x, y, z, mu, phi, stokes, wavelength, fill_ray_variables=True)
pyshdom.checks.check_sensor(sensor)

### The Perspective Sensor

The perspective sensor generator (`pyshdom.sensor.perspective_projection`) is a pinhole sensor and is defined by its location, pointing direction and resolution. This generator also includes explicit method for generating sub-pixel rays to model the FOV.

Note that the attributes contain the input information used to generate this sensor.

In [7]:
sensor = pyshdom.sensor.perspective_projection(0.86, 5.0, 100, 100,
                           [0,0,1], [0,0,0], [0,1,0],
                           stokes=['I'], sub_pixel_ray_args={'method':None})
sensor

<xarray.Dataset>
Dimensions:            (stokes_index: 4, npixels: 10000, image_dims: 2, nrays: 10000)
Coordinates:
  * stokes_index       (stokes_index) <U1 'I' 'Q' 'U' 'V'
  * image_dims         (image_dims) <U2 'nx' 'ny'
Dimensions without coordinates: npixels, nrays
Data variables: (12/16)
    wavelength         float64 0.86
    stokes             (stokes_index) bool True False False False
    cam_x              (npixels) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    cam_y              (npixels) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    cam_z              (npixels) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    cam_mu             (npixels) float64 0.9981 0.9982 0.9982 ... 0.9982 0.9981
    ...                 ...
    ray_x              (nrays) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ray_y              (nrays) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ray_z              (nrays) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    pixel_index        (nrays) int64 0 1 2 3 4 5 ... 9995 9996 9997 9998 9999
    ray_weight         (nrays) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    use_subpixel_rays  bool False
Attributes:
    projection:                         Perspective
    fov_deg:                            5.0
    fov_x_deg:                          5.0
    fov_y_deg:                          5.0
    x_resolution:                       100
    y_resolution:                       100
    position:                           [0. 0. 1.]
    lookat:                             [0. 0. 0.]
    rotation_matrix:                    [-1.  0.  0.  0.  1.  0.  0.  0. -1.]
    sensor_to_camera_transform_matrix:  [22.903765  0.        0.        0.   ...

### Generating sub-pixel-rays

We can pass subpixel ray generators to the sensor generators (`pyshdom.sensor.perspective_projection` and `.orthographic_projection`). In this case we use gauss-legendre quadrature points generated in the image plane of the sensor. The other supported option is stochastic generation.

We use 2nd order gauss-legendre quadrature in both directions of the image plane and we can see that there are now 4 times as many rays as pixels and each ray has a unique pointing angle and weight. These schemes using subpixel rayscan be effective for estimating wide FOV radiances. However, they have ray effects; if the spacing of the rays is not sufficiently dense compared to the RTE solution grid, then information from grid points that are not sampled is lost. This has significant implicatinos for the calculation of the derivatives. 

There is currently no automatic method for determining the requisite number of sub-pixel rays, particularly for the `perspective_projection`, for which it will depend on how far away from the grid the sensor is.

In [8]:
sensor = pyshdom.sensor.perspective_projection(0.86, 5.0, 100, 100,
                           [0,0,1], [0,0,0], [0,1,0],
                           stokes=['I'], sub_pixel_ray_args={'method':pyshdom.sensor.gaussian,
                                                            'degree': (2,2)})
sensor

<xarray.Dataset>
Dimensions:            (stokes_index: 4, npixels: 10000, image_dims: 2, nrays: 40000)
Coordinates:
  * stokes_index       (stokes_index) <U1 'I' 'Q' 'U' 'V'
  * image_dims         (image_dims) <U2 'nx' 'ny'
Dimensions without coordinates: npixels, nrays
Data variables: (12/16)
    wavelength         float64 0.86
    stokes             (stokes_index) bool True False False False
    cam_x              (npixels) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    cam_y              (npixels) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    cam_z              (npixels) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    cam_mu             (npixels) float64 0.9981 0.9982 0.9982 ... 0.9982 0.9981
    ...                 ...
    ray_x              (nrays) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ray_y              (nrays) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ray_z              (nrays) float32 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    pixel_index        (nrays) int64 0 0 0 0 1 1 ... 9998 9999 9999 9999 9999
    ray_weight         (nrays) float64 0.25 0.25 0.25 0.25 ... 0.25 0.25 0.25
    use_subpixel_rays  bool True
Attributes:
    projection:                         Perspective
    fov_deg:                            5.0
    fov_x_deg:                          5.0
    fov_y_deg:                          5.0
    x_resolution:                       100
    y_resolution:                       100
    position:                           [0. 0. 1.]
    lookat:                             [0. 0. 0.]
    rotation_matrix:                    [-1.  0.  0.  0.  1.  0.  0.  0. -1.]
    sensor_to_camera_transform_matrix:  [22.903765  0.        0.        0.   ...
    sub_pixel_ray_args_method:          gaussian
    sub_pixel_ray_args_degree:          (2, 2)

### SensorDict

Groups of sensors which share an uncertainty model are designated as instruments. And groups of instruments are stored in `pyshdom.containers.SensorsDict`, which is a glorified `OrderedDict`. These groups of sensors are labeled. The `pyshdom.containers.SensorsDict` is the container which interacts with the other high level objects. Individual datasets that contain pixel geometries can still be used to directly interface with a `pyshdom.solver.RTE` object to obtain observables at the specified sensor geometry.

In [9]:
sensor_dict = pyshdom.containers.SensorsDict()
sensor_dict.add_sensor('MISR', sensor)
sensor_dict

SensorsDict([('MISR',
              {'sensor_list': [<xarray.Dataset>
                Dimensions:            (stokes_index: 4, npixels: 10000, image_dims: 2, nrays: 40000)
                Coordinates:
                  * stokes_index       (stokes_index) <U1 'I' 'Q' 'U' 'V'
                  * image_dims         (image_dims) <U2 'nx' 'ny'
                Dimensions without coordinates: npixels, nrays
                Data variables: (12/16)
                    wavelength         float64 0.86
                    stokes             (stokes_index) bool True False False False
                    cam_x              (npixels) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
                    cam_y              (npixels) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
                    cam_z              (npixels) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
                    cam_mu             (npixels) float64 0.9981 0.9982 0.9982 ... 0.9982 0.9981
                    ...            

Sensors in the `pyshdom.containers.SensorsDict` object can be directly accessed using the instrument name and the 'sensor_list' string. Note that these are just containers so in-place modifications to `sensor` will also affect what is inside `sensor_dict`.

In [10]:
sensor_dict['MISR']['sensor_list'][0]

<xarray.Dataset>
Dimensions:            (stokes_index: 4, npixels: 10000, image_dims: 2, nrays: 40000)
Coordinates:
  * stokes_index       (stokes_index) <U1 'I' 'Q' 'U' 'V'
  * image_dims         (image_dims) <U2 'nx' 'ny'
Dimensions without coordinates: npixels, nrays
Data variables: (12/16)
    wavelength         float64 0.86
    stokes             (stokes_index) bool True False False False
    cam_x              (npixels) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    cam_y              (npixels) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    cam_z              (npixels) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    cam_mu             (npixels) float64 0.9981 0.9982 0.9982 ... 0.9982 0.9981
    ...                 ...
    ray_x              (nrays) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ray_y              (nrays) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ray_z              (nrays) float32 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    pixel_index        (nrays) int64 0 0 0 0 1 1 ... 9998 9999 9999 9999 9999
    ray_weight         (nrays) float64 0.25 0.25 0.25 0.25 ... 0.25 0.25 0.25
    use_subpixel_rays  bool True
Attributes:
    projection:                         Perspective
    fov_deg:                            5.0
    fov_x_deg:                          5.0
    fov_y_deg:                          5.0
    x_resolution:                       100
    y_resolution:                       100
    position:                           [0. 0. 1.]
    lookat:                             [0. 0. 0.]
    rotation_matrix:                    [-1.  0.  0.  0.  1.  0.  0.  0. -1.]
    sensor_to_camera_transform_matrix:  [22.903765  0.        0.        0.   ...
    sub_pixel_ray_args_method:          gaussian
    sub_pixel_ray_args_degree:          (2, 2)